# Мэтчинг товаров дилера и заказчика

## Описание проекта

Заказчик производит несколько сотен различных товаров бытовой и промышленной химии, а затем продаёт эти товары через дилеров. Дилеры, в свою очередь, занимаются розничной продажей товаров в крупных сетях магазинов и на онлайн
площадках. Для оценки ситуации, управления ценами и бизнесом в целом, заказчик периодически собирает информацию о том, как дилеры продают их товар. Для этого они парсят сайты дилеров, а затем сопоставляют товары и цены. Зачастую описание товаров на сайтах дилеров отличаются от того описания, что даёт заказчик. Например, могут добавляться новый слова (“универсальный”, “эффективный”), объём (0.6 л -> 600 мл). Поэтому сопоставление товаров дилеров с товарами производителя делается вручную.  
Основная идея - предлагать несколько товаров заказчика, которые с наибольшей вероятностью соответствуют размечаемому товару дилера. Предлагается реализовать это решение, как онлайн сервис, открываемый в веб- браузере. Выбор наиболее вероятных подсказок делается методами машинного обучения

**ЦЕЛЬ:** разработать решения, которое автоматизирует процесс сопоставления товаров.  

**Задачи:**
   - выгрузить данные
   - сделать предобработку
   - обработать текст в столбцах с названиями товаров
   - создать ембеддинги предложений 
   - попробовать разные модели,
   - оценить метрики и выбрать лучшую модель


## Описание данных

Заказчик предоставил несколько таблиц (дамп БД), содержащих необходимые данные:  

1. `dealers` - список дилеров:
   - id - уникальный ключ дилера;
   - name - наименование дилера</br>
</br>  
2. `dealer_products` - результат работы парсера площадок дилеров:
   - product_key - уникальный номер позиции;
   - price - цена;
   - product_url - адрес страницы, откуда собраны данные;
   - product_name - заголовок продаваемого товара;
   - date - дата получения информации;
   - dealer_id - идентификатор дилера (внешний ключ к dealers)</br>
</br>
3. `products` - список товаров, которые производит и распространяет заказчик:
   - id - уникальный ключ товара в базе заказчика
   - article - артикул товара;
   - ean_13 - код товара (см. EAN 13)
   - name - название товара;
   - cost - стоимость;
   - recommended_price - рекомендованная цена;
   - category_id - категория товара;
   - ozon_name - названиет товара на Озоне;
   - name_1c - название товара в 1C;
   - wb_name - название товара на Wildberries;
   - ozon_article - описание для Озон;
   - wb_article - артикул для Wildberries;
   - ym_article - артикул для Яндекс.Маркета;</br>  
</br>  
4. `match` - таблица матчинга товаров заказчика и товаров дилеров:
   - key - внешний ключ к dealer_products;
   - product_id - внешний ключ к products;
   - dealer_id - внешний ключ к dealers.

In [70]:
import nltk
import spacy
import re 
import string
import torch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from scipy import sparse
import pickle
import warnings
warnings.simplefilter(action='ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 500)

## Выгрузка и обработка данных

In [2]:
# Выгрузим данные из 4х таблиц в отдельные датафреймы
dealers = pd.read_csv('marketing_dealer.csv', sep=';')
dealer_products = pd.read_csv('marketing_dealerprice.csv', sep=';')
products = pd.read_csv('marketing_product.csv', sep=';')
match = pd.read_csv('marketing_productdealerkey.csv', sep=';')

In [3]:
dealers.sample(5)

,id,name
13,16,Vimos
10,13,simaLand
3,5,Castorama
16,18,Мasterstroy_spb_OZON\r\n
15,8,Leroy_Merlin


In [4]:
dealers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18 non-null     int64 
 1   name    18 non-null     object
dtypes: int64(1), object(1)
memory usage: 416.0+ bytes


In [5]:
dealers['name'].unique()

array(['Moi_vibor_WB', 'Akson', 'Bafus', 'Castorama', 'Cubatora', 'Komus',
       'Megastroy', 'OnlineTrade', 'Petrovich', 'sdvor', 'simaLand',
       'VegosM', 'Vse_instrumeni', 'Vimos', 'Baucenter', 'Leroy_Merlin',
       'Мasterstroy_spb_OZON\r\n', 'Unicleaner_OZON'], dtype=object)

В таблице `dealers` собрана информация по названиям дилеров и их id. Всего в таблице представлено 18 уникальных дилеров.  
Для решения поставленной задачи на текущий момент данная таблица не требуется.

In [6]:
dealer_products.sample(5)

,id,product_key,price,product_url,product_name,date,dealer_id
5524,5659,100121832,1028.0,https://www.bafus.ru/100121832/,Просепт Professional Crystal с Алоэ Вера жидкий моющий концентрат для стирки белья (3 л),2023-07-14,3
12492,12594,857015,371.0,https://akson.ru//p/shpatlevka_zamazka_proseptn_plastix_dlya_zadelki_glubokih_vyboin_i_treschin_1_4kg/,"Шпаклевка выравнивающая акриловая PROSEPT Plastix белая, 1 кг.",2023-07-24,2
14916,14969,26391106,508.0,https://vimos.ru/product/germetik-akrilovyj-prosept-teplyj-sov-oreh-gotovyj-sostav-06-l,Герметик акриловый межшовный для дер. конструкций Prosept орех 0.6л,2023-07-25,16
5687,5821,700000505,319.0,https://baucenter.ru/sredstva-ot-pleseni-gribka/861499/,Средство для удаления плесени PROSEPT 500 мл,2023-07-14,4
8656,8716,100156107,270.0,https://www.bafus.ru/100156107/,Просепт средство для снятия обоев готовый состав (1 л),2023-07-18,3


In [7]:
# удалим лишний столбец 'id
dealer_products = dealer_products.drop(['id'], axis=1)

In [8]:
dealer_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20416 entries, 0 to 20415
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_key   20416 non-null  object 
 1   price         20416 non-null  float64
 2   product_url   20182 non-null  object 
 3   product_name  20416 non-null  object 
 4   date          20416 non-null  object 
 5   dealer_id     20416 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 957.1+ KB


In [9]:
dealer_products.nunique()

product_key     1965
price           1286
product_url     1883
product_name    1953
date              14
dealer_id         18
dtype: int64

In [10]:
sorted(dealer_products['date'].unique())

['2023-07-11',
 '2023-07-12',
 '2023-07-13',
 '2023-07-14',
 '2023-07-17',
 '2023-07-18',
 '2023-07-19',
 '2023-07-21',
 '2023-07-24',
 '2023-07-25',
 '2023-07-26',
 '2023-07-27',
 '2023-07-28',
 '2023-07-31']

В таблице из 20416 записей лишь около 10% уникальных ключей, ссылок и названий продуктов. Все данные собраны за 14 дней: с 11-07 по 31-0-23.

In [11]:
dealer_products.isna().sum()

product_key       0
price             0
product_url     234
product_name      0
date              0
dealer_id         0
dtype: int64

В столбце product_url имеются пропуски, исследуем его подробнее.

In [12]:
no_url = dealer_products[dealer_products['product_url'].isna()]['dealer_id'].unique()
no_url

array([7], dtype=int64)

In [13]:
dealers[dealers['id'] == no_url[0]]

,id,name
5,7,Komus


In [14]:
dealer_products[dealer_products['dealer_id'] == 7]

,product_key,price,product_url,product_name,date,dealer_id
97,1462352,189.0,NaN,Средство для удаления жира и нагара Prosept Cooky Grill 500 мл,2023-07-11,7
177,1462335,233.0,NaN,Полироль для мебели Prosept Universal Polish 500 мл,2023-07-31,7
397,1462337,213.0,NaN,Средство для чистки каминных стекол Prosept Universal Hard 500 мл,2023-07-11,7
689,1565304,149.0,NaN,Средство для мытья пола Prosept Multipower 800 мл,2023-07-11,7
717,1462346,159.0,NaN,Средство для прочистки труб Prosept Bath Prof жидкость 1 л,2023-07-11,7
...,...,...,...,...,...,...
19213,1462347,189.0,NaN,Средство для сантехники Prosept Bath Acryl +акрил 1 л,2023-07-31,7
19214,1462346,159.0,NaN,Средство для прочистки труб Prosept Bath Prof жидкость 1 л,2023-07-31,7
19215,1462352,189.0,NaN,Средство для удаления жира и нагара Prosept Cooky Grill 500 мл,2023-07-31,7
19216,1462354,219.0,NaN,Средство для чистки ковровых покрытий Prosept Carpet DryClean шампунь 500 мл,2023-07-31,7


Все имеющиеся в таблице пропуски относятся к дилеру под номером 7 - Комус.

In [15]:
# Проверим длину названий продуктов для определения неявных пропусков.
dealer_products['product_name'].str.len().min(), dealer_products['product_name'].str.len().max()

(8, 131)

In [16]:
dealer_products[dealer_products['product_name'].str.len() == 8]

,product_key,price,product_url,product_name,date,dealer_id
32,44231946,994.0,https://www.wildberries.ru/catalog/44231946,ОSB BASE,2023-07-11,1
1718,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-11,1
1885,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-11,1
1963,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-11,1
3276,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-12,1
5110,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-13,1
6766,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-14,1
8346,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-17,1
11054,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-19,1
12269,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-21,1


In [17]:
# проверим данные на дубликаты
dealer_products.duplicated().sum()

726

In [18]:
duplicates = dealer_products.duplicated()
dealer_products[duplicates].sort_values(by='product_name').head(5)

,product_key,price,product_url,product_name,date,dealer_id
1945,30420470,263.0,https://www.wildberries.ru/catalog/30420470/detail.aspx?targetUrl=SP,Bath Acid,2023-07-11,1
1867,30420470,263.0,https://www.wildberries.ru/catalog/30420470/detail.aspx?targetUrl=SP,Bath Acid,2023-07-11,1
1866,44231972,325.0,https://www.wildberries.ru/catalog/44231972/detail.aspx?targetUrl=SP,Bath Acid + Концентрат 1 л,2023-07-11,1
1944,44231972,325.0,https://www.wildberries.ru/catalog/44231972/detail.aspx?targetUrl=SP,Bath Acid + Концентрат 1 л,2023-07-11,1
1849,44232028,271.0,https://www.wildberries.ru/catalog/44232028/detail.aspx?targetUrl=SP,Bath Acid Концентрат,2023-07-11,1


In [19]:
# уберём дубликаты по столбцам: ключ, url и названию
dealer_products.drop_duplicates(subset=['product_key', 'product_url', 'product_name'], inplace=True)

In [20]:
dealer_products.duplicated(subset=['product_key', 'product_url', 'product_name']).sum()

0

In [21]:
# сбросим индексы
dealer_products.reset_index(drop = True, inplace = True)

В таблице `dealer_products` 20416 записей.  
Имеются пропуски в столбце `product_url` - 234 записи и все для дилера с id 7 - Komus. Полных дублей в таблице нет, но есть повторяющиейся записи в зависимости от даты выгрузки. 
Все столбцы имеют правильный тип, кроме даты, в рамках проекта дату приводить к нужному формату нет необходимости.  
Столбец `product_key` содержит данные в текстового типа, он состоит из ключей не только в виде числа, но и в виде ссылок на сайты с продуктами.   

Столбец `product_name` является целевым: по нему будем находить соответствие продуктов из базы заказчика.  
В названиях имеются как слова на кириллице, так и на латинице; есть специальные символы, единицы измерения разные: кг, л, мл; в некоторых названиях в конце указан код, состоящий из цифр и "-"; попадаются сокращения (например: дер. конструкций, д/удаления), в рамках одного названия встречаются буквы в разных регистрах.

In [22]:
# удалим лишний столбец 'Unnamed: 0'
products = products.drop(['Unnamed: 0'], axis=1)

In [23]:
products.sample(5)

,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
306,238,083-1,4.610093e+12,"Шпатлевка-замазка для заделки швов по монтажной пене Flastic / 1,4 кг",179.00,374.00,57.0,"Шпатлевка-замазка для заделки швов по монтажной пене PROSEPT Flastic, 1.4 кг.","Шпатлевка-замазка для заделки швов по монтажной пене PROSEPT Flastic, 1.4 кг.","Шпатлевка-замазка для заделки швов по монтажной пене PROSEPT Flastic, 1.4 кг.",469576057.0,151558264.0,083-1,NaN
236,464,М013-12,4.610093e+12,"Герметик акриловый цвет белый , ф/п 600 мл. (12 штук )",3686.40,7716.96,25.0,"Герметик акриловый белый для швов для деревянных домов, конструкций, изделий PROSEPT, ф/п 600 мл. (12 штук )","Герметик акриловый белый для швов для деревянных домов, конструкций, изделий PROSEPT, ф/п 600 мл. (12 штук )","Герметик акриловый белый для швов для деревянных домов, конструкций, изделий PROSEPT, ф/п 600 мл. (12 штук )",189522821.0,161240420.0,M013-12,NaN
421,38,246-1,4.680008e+12,Концентрат для мытья половMULTIPOWER с ароматом цитрусаконцентрат / 1 л,101.94,239.00,50.0,"Средство для мытья полов PROSEPT Multipower citrus, 1л.","Средство для мытья полов PROSEPT Multipower citrus, 1л.","Средство для мытья полов PROSEPT Multipower citrus, 1л.",417953521.0,151231986.0,246-1,NaN
26,55,294-075,4.680008e+12,"Средство усиленного действия для удаления ржавчины и минеральных отложенийBath Acid + с ароматом цитрусаконцентрат 1:200-1:500 / 0,75 л",75.00,176.00,52.0,"Средство PROSEPT усиленного действия для удаления ржавчины и минеральных отложений Bath Acid + с аромат цитруса. Концентрат. 0,75л","Средство PROSEPT усиленного действия для удаления ржавчины и минеральных отложений Bath Acid + с аромат цитруса. Концентрат. 0,75л","Средство PROSEPT усиленного действия для удаления ржавчины и минеральных отложений Bath Acid + с аромат цитруса. Концентрат. 0,75л",413264552.0,149811030.0,294-075,294-0750
94,366,0024-3м,NaN,"Герметик акриловый цвет Медовый, 3 кг",1251.00,2145.00,25.0,NaN,"Герметик акриловый цвет Медовый, 3 кг",NaN,NaN,NaN,NaN,NaN


In [24]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 496 non-null    int64  
 1   article            496 non-null    object 
 2   ean_13             464 non-null    float64
 3   name               494 non-null    object 
 4   cost               491 non-null    float64
 5   recommended_price  491 non-null    float64
 6   category_id        447 non-null    float64
 7   ozon_name          458 non-null    object 
 8   name_1c            485 non-null    object 
 9   wb_name            455 non-null    object 
 10  ozon_article       365 non-null    float64
 11  wb_article         340 non-null    float64
 12  ym_article         337 non-null    object 
 13  wb_article_td      32 non-null     object 
dtypes: float64(6), int64(1), object(7)
memory usage: 54.4+ KB


In [25]:
products.nunique()

id                   496
article              496
ean_13               464
name                 487
cost                 338
recommended_price    319
category_id           38
ozon_name            454
name_1c              473
wb_name              451
ozon_article         365
wb_article           339
ym_article           337
wb_article_td         32
dtype: int64

In [26]:
# удалим столбец wb_article_td так как он содержит мало записей и не содержит важной информации.
products.drop(['wb_article_td'], axis=1, inplace=True)

In [27]:
products.isna().sum()

id                     0
article                0
ean_13                32
name                   2
cost                   5
recommended_price      5
category_id           49
ozon_name             38
name_1c               11
wb_name               41
ozon_article         131
wb_article           156
ym_article           159
dtype: int64

In [28]:
mask = products['name'].isna()
products[mask]

,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article
23,503,0024-7 о,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150126213.0,NaN
35,504,w022-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#удалим строки, где пропуски в названиии товара
products.dropna(subset=['name'], inplace=True)
products.reset_index(drop=True, inplace= True)

In [30]:
mask = products['cost'].isna()
products[mask]

,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article
4,502,0024-7 б,NaN,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN,NaN,189522867.0,150126216.0,0024-7-б
107,449,0024-06 м12,NaN,"Герметик акриловый цвет Медовый 0,6 л (12 шт)",NaN,NaN,25.0,NaN,"Герметик акриловый цвет Медовый 0,6 л (12 шт)",NaN,NaN,NaN,NaN
108,454,0024-06 о12,NaN,"Герметик акриловый цвет Орех, ф/п 600мл (12 штук )",NaN,NaN,25.0,NaN,"Герметик акриловый цвет сосна, ф/п 600мл (12 штук )",NaN,NaN,NaN,NaN


In [31]:
products.duplicated().sum()

0

In [32]:
# изучим наименования продуктов детальнее
products[['name', 'name_1c']].sample(5)

,name,name_1c
475,Средство для устранения засоров в трубахBath Profконцентрат 1:100 / 5 л,"Средство для прочистки труб от засоров PROSEPT Bath Prof, 5 л."
94,"Антисептик ULTRA, концентрат, 1 л, 2 шт","Антисептик ULTRA, концентрат, 1 л, 2 шт"
468,"Удалитель цемента CEMENT CLEANER готовый состав / 0,5 л","Удалитель цемента PROSEPT CEMENT CLEANER, 0.5 л."
386,Краска резиновая серый Ral 7004 / 1 кг,Краска резиновая серый Ral 7004 / 1 кг
317,Средство для чистки акриловых поверхностейBath Acryl концентрат 1:30-1:80 / 1 л,"Средство для чистки акриловых ванн и душевых кабин PROSEPT Bath Acryl, 1 л."


На первый взгляд в названиях из 1С меньше лишней или технической информации, меньше опечаток.

In [33]:
# проверим минимальную и максимальную длину названия
products['name'].str.len().min(), products['name'].str.len().max()

(3, 136)

In [34]:
products[products['name'].str.len() == 3]
# в данной строке отсутствует название, запись можно удалить

,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article
96,436,Р1 09005,4.680008e+12,,500.0,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
i = products[products['name'].str.len() == 3].index
products.drop(i, inplace = True)
products.reset_index(drop = True, inplace = True)

In [36]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 493 non-null    int64  
 1   article            493 non-null    object 
 2   ean_13             463 non-null    float64
 3   name               493 non-null    object 
 4   cost               490 non-null    float64
 5   recommended_price  490 non-null    float64
 6   category_id        447 non-null    float64
 7   ozon_name          458 non-null    object 
 8   name_1c            485 non-null    object 
 9   wb_name            455 non-null    object 
 10  ozon_article       365 non-null    float64
 11  wb_article         339 non-null    float64
 12  ym_article         337 non-null    object 
dtypes: float64(6), int64(1), object(6)
memory usage: 50.2+ KB


В таблице `products` 496 записей.  
Имеются 2 записи, где отсутствует большая часть информации.  
Для товаров *Герметик акриловой цвет Белый, 7 кг; Герметик акриловый цвет Медовый 0,6 л (12 шт); Герметик акриловый цвет Орех, ф/п 600мл (12 штук)* отсутствуют стоимость и рекомендованная цена.
Дубликаты отсутствуют.  

Для построения модели мэчинга можем использовать данные в столбце `name` или `name_1c`, всего имеется 487 уникальных наименований.
В названиях имеются опечатки, лишние пробелы, специальные символы, иногда отсутствуют пробелы между словами: часто сливаются слова на кириллице и латинице. В части продуктов указана рекомендуемая концентрация, для некоторых продуктов указан вес (в кг.), а для других объём (в мл. или л.). Концентрация, вес или количество обычно указываются в конце названия. В рамках одного названия встречаются буквы в разных регистрах. Максимальная длина наименования продукта 136 символов, минимальная - 30.

In [37]:
# удалим лишний столбец 'Unnamed: 0'
match = match.drop(['id'], axis = 1)

In [38]:
match.sample(5)

,key,dealer_id,product_id
362,200544060,3,25
155,100156057,3,177
1434,1001472246,5,307
369,100121743,3,204
1529,970372102,17,267


In [39]:
match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   key         1700 non-null   object
 1   dealer_id   1700 non-null   int64 
 2   product_id  1700 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 40.0+ KB


In [40]:
match.nunique()

key           1700
dealer_id       18
product_id     438
dtype: int64

In [41]:
match.sort_values(by='key', ascending=False)

,key,dealer_id,product_id
440,https://kub02.ru/catalog/prosept/otbelivatel_dlya_drevesiny_prosept_50_1_1_1l/,6,240
439,https://kub02.ru/catalog/prosept/maslo_dlya_zashchity_polkov_prosept_sauna_oil_gotovyy_sostav_0_25l/,6,320
434,https://kub02.ru/catalog/prosept/lak_dlya_bani_i_sauny_termostoykiy_akrilovyy_prosept_0_9l/,6,321
449,https://kub02.ru/catalog/prosept/antiseptik_universalnyy_protiv_gribka_i_pleseni_prosept_antiplesen_got_sostav_5l/,6,290
432,https://kub02.ru/catalog/prosept/antiseptik_universalnyy_dlya_vnutr_i_naruzhn_prosept_universal_1l/,6,259
...,...,...,...
383,100067710,3,291
224,100067709,3,289
175,100067708,3,397
170,100067707,3,242


In [42]:
match.duplicated().sum()

0

В таблице `match` 1700 записей, кол-во уникальных id дилеров совпадает с количеством в таблице `dealers`. Пропусков нет, дубликатов тоже. Столбец `key` имеет текстовый формат, в нём есть записи не только уникальных ключей, но и url продуктов.  
Данная таблица может пригодиться, когда будем оценивать эффективность мэтчинга.

## Предобработка названий

In [43]:
# заполним пропуски в name_1c данными из столбца name
products['name_1c'].fillna(products['name'], inplace=True)

В дальнейшем будем работать со столбцами `1c_name` из таблицы `products` и `product_name` из `dealer_products`.
Необходимо произвести предобработку текста, чтобы названия в обеих таблицах были наиболее схожи, для этого необходимо:
1. убрать лишние пробелы
2. привести к нижнему регистру
3. добавить пробелы между русскими словами и английскими: как до, так и после
4. убрать концентрацию, оставить только объём/вес
5. убрать стоп-слова

In [44]:
# функция для базовой обработки текста
def clean_text(text):
    #добавляем пробелы между русскими и английскими словами
    #pattern = re.compile(r'(?<=[а-яА-Я])(?=[a-zA-Z])|(?<=[a-zA-Z])(?=[а-яА-Я])')
    text = re.sub(r'(?<=[а-яА-Я])(?=[a-zA-Z])|(?<=[a-zA-Z])(?=[а-яА-Я])', ' ', text)
    #убираем указание концентрации
    #pattern2 = re.compile(r'\b\d+:\d+\s*-\s*\d+:\d+\b|\s*\d+:\d+\s*')
    text = re.sub(r'\b\d+:\d+\s*-\s*\d+:\d+\b|\s*\d+:\d+\s*', '', text)
    #убираем специальные символы
    remove = string.punctuation
    #remove = remove.replace("-", "") # не убираем дефисы
    text = re.sub('[%s]' % re.escape(remove), ' ', text)
    # убираем лишние пробелы между словами
    text = re.sub(r'\s+', ' ', text)
    #приводим все слова к нижнему регистру
    #text = text.lower()
    return text

# функция убирает служебные слова и лемматизирует текст
def preprocess_text(text):
    # удаление одиноко стоящих слов
    #text = re.sub(r'\s+[a-zA-Zа-яА-Я0-9]\s+', ' ', text)
    #Очистка текста 
    text = re.sub(r"[^a-zA-Zа-яА-ЯёЁ ]", ' ', text)
    #doc = nlp(text)
    #токенизация, лемматизация 
    tokens = word_tokenize(text.lower())
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    #удаление стоп-слов
    stop_words = set(stopwords.words('russian') + stopwords.words('english'))
    lemmas_clean = [lemma for lemma in lemmas if lemma not in stop_words]
    #tokens = [token.lemma_ for token in doc]
    #tokens = [token for token in tokens if token not in stop_words and token != " "]
    #text = " ".join(tokens)
    return " ".join(lemmas_clean)

# функция выделеяет единицы измерения из текста
def extract_measure(text):
    measurements = []
    pattern = r'\s*(\d+(?:[,.]\d+)?)\s?[л|мл|кг]+'
    match = re.search(pattern, text)
    if match:
        measurements =  match.group(1)
        text = text.replace(pattern, '')
        text = text.replace(' ', '')
    else:
        measurements = 0
        text = text.replace(pattern, '')
        text = text.replace(' ', '')
    return measurements

# функция вовращает список длин
def get_text_length(x):
    return np.array([len(t) for t in x]).reshape(-1, 1)

In [45]:
# создадим столбцы с единицами измерения товара в обеих таблицах
products['measures'] = products['name_1c'].apply(extract_measure)
products['measures'] = products['measures'].str.replace(',', '.')
products['measures'].fillna(0, inplace=True)
products['measures'] = products['measures'].astype(float)

dealer_products['measures'] = dealer_products['product_name'].apply(extract_measure)
dealer_products['measures'] = dealer_products['measures'].str.replace(',', '.')
dealer_products['measures'].fillna(0, inplace=True)
dealer_products['measures'] = dealer_products['measures'].astype(float)

In [46]:
products.sample(5)

,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,measures
408,246,008-5,4.680008e+12,Антисептик невымываемыйPROSEPT ULTRAконцентрат 1:10 / 5 л,1729.00,4121.0,20.0,"Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 5 л.","Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 5 л.","Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 5 л.",189522714.0,150033486.0,008-5,5.0
317,378,047-3,4.680008e+12,"Грунт АКВАИЗОЛ, голубой, концентрат 1:4 / 3 л",475.00,1071.0,26.0,"Грунт влагоизолирующий PROSEPT Акваизол, 3 л.","Грунт влагоизолирующий PROSEPT Акваизол, 3 л.","Грунт влагоизолирующий PROSEPT Акваизол, 3 л.",453027209.0,149699636.0,047-3,3.0
354,413,066-10,4.680008e+12,"Антисептик универсальный ХМФ-БФ, ГОСТ / 10 л",857.00,1794.0,20.0,"Антисептик универсальный PROSEPT ХМФ-БФ ГОСТ, 10 л.","Антисептик универсальный PROSEPT ХМФ-БФ ГОСТ, 10 л.","Антисептик универсальный PROSEPT ХМФ-БФ ГОСТ, 10 л.",253565301.0,150033496.0,066-10,10.0
356,265,009-1,4.680008e+12,Антисептик для влажной древесиныPROSEPT BiOконцентрат 1:19 / 1 л,274.00,651.0,20.0,"Антисептик для влажной древесины PROSEPT BiO, концентрат, 1 л.","Антисептик для влажной древесины PROSEPT BiO, концентрат, 1 л.","Антисептик для влажной древесины PROSEPT BiO, концентрат, 1 л.",452576233.0,150033505.0,009-1,1.0
197,482,М034-2,4.610093e+12,"Набор для бани (Universal Wood, Multipower Wood)",218.66,491.0,NaN,"Набор для бани (Universal Wood, Multipower Wood)","Набор для бани (Universal Wood, Multipower Wood)","Набор для бани (Universal Wood, Multipower Wood)",NaN,164417207.0,NaN,0.0


In [47]:
#создадим столбцы с длиной названия товара
products['name_length'] = get_text_length(np.array(products['name_1c']))
dealer_products['name_length'] = get_text_length(np.array(dealer_products['product_name']))

In [48]:
%%time
# создадим новый столбец marketing_name - он включает в себя все названия из 1с
products['marketing_name'] = products['name_1c'].apply(clean_text)
#pattern = r'\b(\d+)\s?[л|мл|кг]+'
products['marketing_name'] = products['marketing_name'].str.replace(r'\b(\d+)\s?[л|мл|кг]+', '')

CPU times: total: 1.33 s
Wall time: 1.33 s


In [49]:
products[['name_1c','marketing_name']].sample(3)

,name_1c,marketing_name
137,Средство для мытья посуды в посудомоечной машине. Для жесткой водыCooky Splash Softконцентрат 1:200-1:2000 / 20 л,Средство для мытья посуды в посудомоечной машине Для жесткой воды Cooky Splash Soft концентрат
145,"Антисептик лессирующийзащитно-декоративныйPROSEPT BiO LASUR / бесцветный / 2,7 л",Антисептик лессирующийзащитно декоративный PROSEPT BiO LASUR бесцветный 2
129,"Антисептик SAUNA, концентрат, 1 л, 2 шт",Антисептик SAUNA концентрат 2 шт


In [50]:
%%time
# лемматизируем текст
#nlp = spacy.load("ru_core_news_lg")
products['marketing_name'] = products['marketing_name'].apply(preprocess_text)

CPU times: total: 1.59 s
Wall time: 1.59 s


In [51]:
products[['name_1c','marketing_name']].sample(3,random_state=1)

,name_1c,marketing_name
354,"Антисептик универсальный PROSEPT ХМФ-БФ ГОСТ, 10 л.",антисептик универсальный prosept хмф бф гост
107,"Герметик акриловый цвет сосна, ф/п 600мл (12 штук )",герметик акриловый цвет сосна ф п штук
165,Гель эконом-класса для мытья посуды вручную. Без запахаCooky Е концентрированное средство / 5 л ПЭТ,гель эконом класса мытья посуды вручную запаха cooky е концентрированное средство пэт


In [52]:
dealer_products['product_name'].head()

0           Средство универсальное Prosept Universal Spray, 500мл
1        Концентрат Prosept Multipower для мытья полов, цитрус 1л
2    Средство для чистки люстр Prosept Universal Anti-dust, 500мл
3             Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 023-05
4     Средство моющее для бани и сауны Prosept Multipower Wood 1л
Name: product_name, dtype: object

In [53]:
%%time
# аналогичным образом обработаем столбец product_name
dealer_products['dealer_name'] = dealer_products['product_name'].apply(clean_text)
dealer_products['dealer_name'] = dealer_products['dealer_name'].str.replace(r'\b(\d+)\s?[л|мл|кг]+', '')
dealer_products['dealer_name'].head()

CPU times: total: 93.8 ms
Wall time: 89 ms


0              Средство универсальное Prosept Universal Spray 
1        Концентрат Prosept Multipower для мытья полов цитрус 
2       Средство для чистки люстр Prosept Universal Anti dust 
3            Удалитель ржавчины PROSEPT RUST REMOVER 0  023 05
4    Средство моющее для бани и сауны Prosept Multipower Wood 
Name: dealer_name, dtype: object

In [54]:
%%time
dealer_products['dealer_name'] = dealer_products['dealer_name'].apply(preprocess_text)
dealer_products['dealer_name'].head()

CPU times: total: 1.27 s
Wall time: 1.28 s


0        средство универсальное prosept universal spray
1      концентрат prosept multipower мытья полов цитрус
2     средство чистки люстр prosept universal anti dust
3               удалитель ржавчины prosept rust remover
4    средство моющее бани сауны prosept multipower wood
Name: dealer_name, dtype: object

In [55]:
# объединим id, marketing_name, product_key, dealer_name с таблицей match
match_products = match[['key', 'product_id']].merge(
    products[['id', 'marketing_name']],  how='left', right_on='id', left_on='product_id')

# к датасету с названиями диллеров присоединим названия производителя 
df = dealer_products[['product_key', 'dealer_name']].merge(
    match_products, how='left', right_on='key', left_on='product_key').drop(['product_id', 'product_key'], axis=1)

df = df.dropna().reset_index(drop=True)
df

,dealer_name,key,id,marketing_name
0,средство универсальное prosept universal spray,546227,12.0,универсальное чистящее средство prosept universal spray
1,концентрат prosept multipower мытья полов цитрус,546408,38.0,средство мытья полов prosept multipower citrus
2,средство чистки люстр prosept universal anti dust,546234,18.0,несмываемое средство очистки люстр prosept universal anti dust
3,удалитель ржавчины prosept rust remover,651258,403.0,удалитель ржавчины prosept rust remover
4,средство моющее бани сауны prosept multipower wood,546355,39.0,моющее средство бани сауны prosept multipower wood
...,...,...,...,...
1752,средство мытья плитки керамогранита prosept multipower kerama,1462340,40.0,средство мытья плитки керамогранита prosept multipower kerama
1753,cement cleaner удалитель цемента,40019201,400.0,удалитель цемента prosept cement cleaner
1754,антисептик невымываемый конструкций,45316302,271.0,невымываемый антисептик ответственных конструкций prosept eco ultra
1755,diona гель перламутром концентрат,44231996,172.0,жидкое мыло перламутром ароматизаторов prosept diona


## Векторизация текста и поиск мэтчей

### SBERT

In [56]:
# %%time
# model = SentenceTransformer('all-MiniLM-L6-v2')

# products = list(df['marketing_name'])
# dealers = df['dealer_name']
# prod_embeddings = model.encode(products)
# deal_embeddings = model.encode(dealers)

In [57]:
# top_k = 5
# cos_scores = util.cos_sim(deal_embeddings, prod_embeddings)
# top_results = torch.topk(cos_scores, k=top_k)
# print("Cosine-Similarity:", cos_sim)

In [58]:
# x = util.semantic_search(prod_embeddings, deal_embeddings, top_k = top_k)
# x

In [59]:
# match_key = []
# metrics = []
# for i in range(len(x[:3])):
#     for k in range(top_k):
#         match_key.append(df.iloc[x[i][k]['corpus_id'],:]['id'])
#         if df.iloc[i]['id'] in match_key[0]:
#             print('yes')

In [60]:
# for query in dealers:
#     # We use cosine-similarity and torch.topk to find the highest 5 scores
#     cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
#     top_results = torch.topk(cos_scores, k=top_k)

#     print("\n\n======================\n\n")
#     print("Query:", query)
#     print("\nTop 5 most similar sentences in corpus:")

#     for score, idx in zip(top_results[0], top_results[1]):
#         print(corpus[idx], "(Score: {:.4f})".format(score))

In [61]:
# for i in range(len(x)):
#     print('===================')
#     print(f"Запрос: {df['dealer_name'][i]}")
#     print('===================')
#     for k in range(3):
#         #print(sentences[x[i][k]['corpus_id']])
#         print(f"Название продукта: {df['marketing_name'][x[i][k]['corpus_id']]}, оценка: {x[i][k]['score']}")

In [62]:
# matches = []

# for col in set(match_df.columns):
        
#     top_cands = match_df.loc[:, col].sort_values(ascending=True)[:top].index.tolist()
#     product_key = ''.join(col.split('_')[:-1])
    
#     # print(product_key)
    
#     if  match.loc[match['key'] == product_key].shape[0] == 0:
#         matches.append(0)
#         continue
        
#     match_id = match.loc[match['key'] == product_key, 'product_id'].values[0]
#     if match_id in top_cands:
#         matches.append(1)
#     else:
#         matches.append(0)
        
# print(f'Значение метрики Accuracy@{top_k} рекомендаций: {np.mean(matches)}')

### Вариант2: 

In [63]:
#Создадим словарь,где ключом являются id продукта, а значением - название
#marketing_name = pd.Series(products['marketing_name'].values, index=products['id']).to_dict()
# marketing_name = pd.Series(products['name_1c'].values, index=products['id']).to_dict()
# marketing_name

In [64]:
#Создадим словарь,где ключом являются product_key дилера, а значением - название у дилера
#dealer_name = pd.Series(dealer_products['dealer_name'].values, index=dealer_products['product_key']).to_dict()
# dealer_name = pd.Series(dealer_products['product_name'].values, index=dealer_products['product_key']).to_dict()
# dealer_name

In [65]:
%%time
model = SentenceTransformer('all-MiniLM-L6-v2')
#model = SentenceTransformer('distilbert-base-nli-mean-tokens') 

rows = df['marketing_name'].values
columns = df['dealer_name'].values
market_names = model.encode(rows)
dealer_names = model.encode(columns)

CPU times: total: 57min 42s
Wall time: 35min 34s


In [80]:
# для оперативности сохраняем готовые эмбеддинги локально
# with open('prod_emb.pickle', 'wb') as f:
#     pickle.dump(market_names, f)

#код для их загрузки
# with open('prod_emb.pickle', 'rb') as f:
#     prod_emb = pickle.load(f)

In [81]:
# with open('deal_emb.pickle', 'wb') as f:
#     pickle.dump(dealer_names, f)
    
# with open('deal_emb.pickle', 'rb') as f:
#     deal_emb = pickle.load(f)

In [66]:
# # дополним векторы названий данными об объёме/весе и длине строки
# X1 = pd.DataFrame(market_names)
# X1['measures'] = products['measures']
# X1['name_length'] = products['name_length']

# X2 = pd.DataFrame(dealer_names)
# X2['measures'] = dealer_products['measures']
# X2['name_length'] = dealer_products['name_length']

# # # отмасштабируем данные
# numeric = ['measures', 'name_length']
# scaler = MinMaxScaler()

# scaler.fit(X1[numeric])
# X1[numeric] = scaler.transform(X1[numeric])

# scaler.fit(X2[numeric])
# X2[numeric] = scaler.transform(X2[numeric])

# # # преобразуем данные в разряженную матрицу
# X1_sparse = sparse.csr_matrix(X1.values)
# X2_sparse = sparse.csr_matrix(X2.values)

In [ ]:
data = pairwise_distances(market_names, dealer_names, metric = 'cosine')

In [68]:
#создадим матрицу соответствий названий
# data = pairwise_distances(X1_sparse, X2_sparse, metric = 'cosine')
# match_df = pd.DataFrame(index = products['name_1c'], columns = dealer_products['product_name'], data=data)
# match_df

data = pairwise_distances(market_names, dealer_names, metric = 'cosine')
match_df = pd.DataFrame(index = df['id'], 
                        columns = df['key']+ '_' + pd.Series(range(df.shape[0])).astype(str), 
                        data=data)
match_df

,546227_0,546408_1,546234_2,651258_3,546355_4,831859_5,546406_6,831858_7,857015_8,651265_9,...,531730388_1747,1092966_1748,44231991_1749,674702000_1750,674682694_1751,1462340_1752,40019201_1753,45316302_1754,44231996_1755,860509000_1756
id,,,,,,,,,,,,,,,,,,,,,
12.0,0.023688,0.692714,0.556602,0.580364,0.725108,0.647015,0.715158,0.647015,0.707425,0.511298,...,0.696416,0.672605,0.816590,0.722360,0.697642,0.715158,0.640365,0.806697,0.722378,0.735137
38.0,0.524135,0.368915,0.669530,0.514967,0.422392,0.478108,0.417285,0.478108,0.525890,0.533097,...,0.494099,0.615209,0.685083,0.489390,0.466485,0.417285,0.634883,0.705884,0.593594,0.533106
18.0,0.544454,0.753340,0.043476,0.680014,0.739828,0.675381,0.746569,0.675381,0.694778,0.725450,...,0.693059,0.655508,0.635691,0.739133,0.736979,0.746569,0.673864,0.710789,0.807284,0.727109
403.0,0.499496,0.537671,0.568043,0.000000,0.500438,0.477907,0.560491,0.477907,0.439813,0.394115,...,0.406909,0.452221,0.609707,0.464204,0.481138,0.560491,0.389900,0.580768,0.515609,0.486173
39.0,0.656498,0.399492,0.657324,0.510046,0.001966,0.610794,0.409952,0.610794,0.495756,0.484200,...,0.465468,0.521514,0.585476,0.495486,0.507398,0.409952,0.616039,0.695952,0.680694,0.560856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40.0,0.654139,0.156178,0.630230,0.560491,0.405059,0.447868,0.000000,0.447868,0.406018,0.477063,...,0.395368,0.439633,0.570494,0.419534,0.409853,0.000000,0.590515,0.531010,0.446516,0.454909
400.0,0.462567,0.460959,0.540218,0.312035,0.534991,0.476548,0.524963,0.476548,0.509609,0.427598,...,0.485347,0.421167,0.570645,0.519091,0.522729,0.524963,0.044145,0.609502,0.513137,0.440811
271.0,0.633726,0.548530,0.646514,0.572904,0.578185,0.556453,0.553069,0.556453,0.558007,0.545026,...,0.487668,0.441423,0.665371,0.491587,0.486341,0.553069,0.759365,0.472679,0.670333,0.552959


In [69]:
# рассчитаем Accuracy@k для оценки качества мэтчинга
top_k = 5
matches = []

for col in set(match_df.columns):
        
    top_cands = match_df.loc[:, col].sort_values(ascending=True)[:top_k].index.tolist()
    product_key = ''.join(col.split('_')[:-1])
    
    if  match.loc[match['key'] == product_key].shape[0] == 0:
        matches.append(0)
        continue
        
    match_id = match.loc[match['key'] == product_key, 'product_id'].values[0]
    if match_id in top_cands:
        matches.append(1)
    else:
        matches.append(0)
        
np.mean(matches)

0.9823562891291975

### Вариант 3

In [ ]:
# from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.svm import LinearSVC
# from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
# from sklearn.preprocessing import FunctionTransformer
# from collections import defaultdict

# def get_text_length(x):
#     return np.array([len(t) for t in x]).reshape(-1, 1)

In [ ]:
# %%time
# corpus = pd.concat([products['marketing_name'], dealer_products['dealer_name']], axis = 0)
# count_tf_idf = TfidfVectorizer()
# corpus_vect = count_tf_idf.fit(corpus)
# df_1 = count_tf_idf.transform(products['marketing_name'])
# df_2 = count_tf_idf.transform(dealer_products['dealer_name'])

In [ ]:
# df_1 = df_1.todense() #df_1.toarray()
# df_2 = df_2.todense() #df_2.toarray()

In [ ]:
# matr = pd.DataFrame(data = pairwise_distances(df_1, df_2, 'cosine'), 
#              index = products['id'], 
#              columns = dealer_products['product_name'])

In [ ]:
# pipe = Pipeline([('count', CountVectorizer(ngram_range= (1,2))),
#                  ('tfid', TfidfTransformer())]).fit(corpus)
# X = pipe['count'].transform(corpus).toarray()
# X = pipe['tfid'].idf_
# X = pipe.transform(corpus)
# X